In [1]:
import sys

In [2]:
import pandas
import requests
import comtradeapicall

In [3]:
from datetime import date
from datetime import timedelta
today = date.today()
yesterday = today - timedelta(days=1)
lastweek = today - timedelta(days=7)

In [6]:
mydf = comtradeapicall.previewFinalData(typeCode='C', freqCode='M', clCode='HS', period='202301',
                                        reporterCode='579', cmdCode='2710', flowCode='X', partnerCode=699,
                                        partner2Code=699,
                                        customsCode=None, motCode=0, maxRecords=500, format_output='JSON',
                                        aggregateBy=None, breakdownMode='plus', countOnly=None, includeDesc=True)

In [17]:
import psycopg2
from psycopg2 import Error
dbname = "postgres"
user = "postgres"
password = "Major"
host = "localhost"
port = "5432"

In [18]:
try:
    connection = psycopg2.connect(
        dbname=dbname,
        user=user,
        password=password,
        host=host,
        port=port
    )
    
    cursor = connection.cursor()
    
    print(connection.get_dsn_parameters(), "\n")

    cursor.execute("SELECT version();")
    record = cursor.fetchone()
    print("You are connected to - ", record, "\n")

except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)

{'user': 'postgres', 'channel_binding': 'prefer', 'dbname': 'postgres', 'host': 'localhost', 'port': '5432', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'sslcertmode': 'allow', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'prefer', 'krbsrvname': 'postgres', 'gssdelegation': '0', 'target_session_attrs': 'any', 'load_balance_hosts': 'disable'} 

You are connected to -  ('PostgreSQL 16.2 (Postgres.app) on aarch64-apple-darwin21.6.0, compiled by Apple clang version 14.0.0 (clang-1400.0.29.102), 64-bit',) 



In [20]:
try:
    create_table_query = '''
    CREATE TABLE IMS_db (RefPeriodId INT, RefYear INT, RefMonth INT, Period INT, ReporterCode INT, ReporterISO VARCHAR(3), ReporterDesc VARCHAR(255), FlowDesc VARCHAR(50), PartnerCode INT, PartnerISO VARCHAR(3), PartnerDesc VARCHAR(255), CmdCode INT, CmdDesc VARCHAR(255), QtyUnitAbbr VARCHAR(10), Qty NUMERIC, NetWgt NUMERIC, PrimaryValue NUMERIC
    );
    '''
    cursor.execute(create_table_query)
    connection.commit()
    print("Table IMS_db created successfully in PostgreSQL")

except (Exception, Error) as error:
    print("Error while creating PostgreSQL table", error)


Table IMS_db created successfully in PostgreSQL


In [21]:
try:
    records = df.values.tolist()
    insert_query = "INSERT INTO IMS_db VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

    cursor.executemany(insert_query, records)
    connection.commit()
    print("Data inserted successfully into PostgreSQL")

except (Exception, Error) as error:
    print("Error while inserting data into PostgreSQL", error)

finally:
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

Data inserted successfully into PostgreSQL
PostgreSQL connection is closed
